# Σημασιολογικός Πυρήνας

Σε αυτό το παράδειγμα κώδικα, θα χρησιμοποιήσετε το [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI Framework για να δημιουργήσετε έναν βασικό πράκτορα.

Ο στόχος αυτού του παραδείγματος είναι να σας δείξει τα βήματα που θα χρησιμοποιήσουμε αργότερα στα επιπλέον παραδείγματα κώδικα κατά την υλοποίηση των διαφορετικών προτύπων πρακτόρων.


## Εισαγωγή των Απαραίτητων Πακέτων Python


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Δημιουργία του Πελάτη

Σε αυτό το παράδειγμα, θα χρησιμοποιήσουμε τα [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) για πρόσβαση στο LLM.

Το `ai_model_id` ορίζεται ως `gpt-4o-mini`. Δοκιμάστε να αλλάξετε το μοντέλο σε κάποιο άλλο διαθέσιμο στην αγορά των GitHub Models για να δείτε διαφορετικά αποτελέσματα.

Για να χρησιμοποιήσουμε το `Azure Inference SDK`, που χρησιμοποιείται για το `base_url` των GitHub Models, θα χρησιμοποιήσουμε τον συνδετήρα `OpenAIChatCompletion` μέσα στο Semantic Kernel. Υπάρχουν επίσης άλλοι [διαθέσιμοι συνδετήρες](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) για τη χρήση του Semantic Kernel με άλλους παρόχους μοντέλων.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Δημιουργία του Agent

Παρακάτω δημιουργούμε τον Agent με το όνομα `TravelAgent`.

Για αυτό το παράδειγμα, χρησιμοποιούμε πολύ απλές οδηγίες. Μπορείτε να αλλάξετε αυτές τις οδηγίες για να δείτε πώς ο agent ανταποκρίνεται διαφορετικά.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Εκτέλεση του Agent

Τώρα μπορούμε να εκτελέσουμε τον Agent ορίζοντας ένα νήμα τύπου `ChatHistoryAgentThread`. Οποιαδήποτε απαραίτητα μηνύματα συστήματος παρέχονται στο όρισμα `messages` της μεθόδου invoke_stream του agent.

Αφού οριστούν αυτά, δημιουργούμε ένα `user_inputs`, το οποίο θα είναι αυτό που στέλνει ο χρήστης στον agent. Σε αυτή την περίπτωση, έχουμε ορίσει αυτό το μήνυμα ως `Σχεδίασέ μου διακοπές με ήλιο`.

Μη διστάσετε να αλλάξετε αυτό το μήνυμα για να δείτε πώς αποκρίνεται διαφορετικά ο agent.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία μετάφρασης AI [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτοματοποιημένες μεταφράσεις ενδέχεται να περιέχουν σφάλματα ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
